In [ ]:
# 뉴스분석 API를 활용하여 웹3 산업 관련 뉴스 인용문 및 키워드 수집
import requests
import json

# API 키와 요청 파라미터 설정
api_key = "f107ffb99e2b44b5855a322c8326efc0"
params = {
    "q": "메타버스", # 검색어
    "sortBy": "relevance", # 정렬 방식
    "language": "ko", # 언어
    "pageSize": 100, # 페이지당 결과 개수
    "apiKey": api_key # API 키
}

# API 요청 및 응답 저장
response = requests.get("https://newsapi.org/v2/everything", params=params)
data = response.json()

# 응답 데이터에서 뉴스 인용문과 키워드 추출
quotes = [] # 인용문 리스트
keywords = [] # 키워드 리스트

for article in data["articles"]:
    # 인용문 추출
    quote = article["content"]
    if quote:
        quotes.append(quote)
    
    # 키워드 추출
    keyword = article["title"]
    if keyword:
        keywords.append(keyword)

# 인용문과 키워드 출력
print("인용문 개수:", len(quotes))
print("키워드 개수:", len(keywords))


In [ ]:
# 수집된 텍스트 데이터를 전처리하여 불필요한 문자, 공백 등을 제거하고, 형태소 분석에 필요한 데이터만 추출
import re

# 전처리 함수 정의
def preprocess(text):
    # 불필요한 문자 제거
    text = re.sub(r"[^가-힣A-Za-z0-9 ]", "", text)
    # 공백 제거
    text = re.sub(r"\s+", " ", text)
    # 양쪽 공백 제거
    text = text.strip()
    return text

# 전처리된 인용문과 키워드 리스트 생성
preprocessed_quotes = []
preprocessed_keywords = []

for quote in quotes:
    preprocessed_quotes.append(preprocess(quote))

for keyword in keywords:
    preprocessed_keywords.append(preprocess(keyword))

# 전처리된 인용문과 키워드 출력
print("전처리된 인용문 개수:", len(preprocessed_quotes))
print("전처리된 키워드 개수:", len(preprocessed_keywords))

In [ ]:
# 추출된 데이터에 대해 형태소 분석기를 적용하여, 단어를 형태소 단위로 분리하고, 각 형태소에 대한 품사 태그를 부착
from konlpy.tag import Okt

# 형태소 분석기 객체 생성
okt = Okt()

# 형태소 분석 결과를 저장할 리스트 생성
okt_quotes = []
okt_keywords = []

# 인용문에 대해 형태소 분석 수행
for quote in preprocessed_quotes:
    okt_quotes.append(okt.pos(quote))

# 키워드에 대해 형태소 분석 수행
for keyword in preprocessed_keywords:
    okt_keywords.append(okt.pos(keyword))

# 형태소 분석 결과 출력
print("Okt 인용문 개수:", len(okt_quotes))
print("Okt 키워드 개수:", len(okt_keywords))

In [ ]:
# 형태소만 저장할 리스트 생성
okt_quotes_morphs = []
okt_keywords_morphs = []

# okt_quotes의 원소들을 반복문으로 순회
for quote in okt_quotes:
    # 형태소만 저장할 임시 리스트 생성
    temp = []
    # quote가 튜플 형태로 되어 있는 형태소와 품사 태그를 분리
    for morph, tag in quote:
        # 형태소만 임시 리스트에 추가
        temp.append(morph)
    # 임시 리스트를 okt_quotes_morphs에 추가
    okt_quotes_morphs.append(temp)

for keyword in okt_keywords:
    # 형태소만 저장할 임시 리스트 생성
    temp = []
    # quote가 튜플 형태로 되어 있는 형태소와 품사 태그를 분리
    for morph, tag in keyword:
        # 형태소만 임시 리스트에 추가
        temp.append(morph)
    # 임시 리스트를 okt_quotes_morphs에 추가
    okt_keywords_morphs.append(temp)

# 결과 출력
print("Okt 인용문 형태소 개수:", len(okt_quotes_morphs))
print("Okt 인용문 형태소:", okt_quotes_morphs)
print("Okt 키워드 형태소 개수:", len(okt_keywords_morphs))
print("Okt 키워드 형태소:", okt_keywords_morphs)

In [ ]:
# 감정 분석을 위한 학습 데이터 불러오기
# 학습 데이터는 네이버 영화 리뷰 데이터를 사용하였습니다.
# 출처: https://github.com/e9t/nsmc
import pandas as pd

train_data = pd.read_csv("ratings_train.txt", sep="\t")
test_data = pd.read_csv("ratings_test.txt", sep="\t")
train_data = train_data.dropna(subset=["document"]) # document 열에 결측치가 있는 행을 제거
test_data = test_data.dropna(subset=["document"])

# 학습 데이터의 형태소 분석 결과를 저장할 리스트 생성
train_okt = []
test_okt = []

# 학습 데이터에 대해 형태소 분석 수행
for review in train_data["document"]:
    train_okt.append(okt.morphs(review))

for review in test_data["document"]:
    test_okt.append(okt.morphs(review))

# 형태소 분석 결과를 정수 인덱스로 변환하기 위한 사전 생성
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer()
tokenizer.fit_on_texts(train_okt)

# 사전에 없는 단어는 0으로 처리하기 위해 인덱스를 1부터 시작하도록 설정
tokenizer.word_index = {key: value + 1 for key, value in tokenizer.word_index.items()}
tokenizer.word_index["<PAD>"] = 0 # 패딩을 위한 특수 토큰

# 형태소 분석 결과를 정수 인덱스로 변환
train_sequences = tokenizer.texts_to_sequences(train_okt)
test_sequences = tokenizer.texts_to_sequences(test_okt)

# 정수 인덱스로 변환된 데이터의 길이를 통일하기 위해 패딩 추가
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_length = 30 # 최대 길이 설정
train_padded = pad_sequences(train_sequences, maxlen=max_length, padding="post")
test_padded = pad_sequences(test_sequences, maxlen=max_length, padding="post")

# 학습 데이터의 레이블(감성)을 numpy 배열로 변환
import numpy as np

train_labels = np.array(train_data["label"])
test_labels = np.array(test_data["label"])

In [ ]:
# LSTM 모델을 사용하여 감정 분석 수행
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

vocab_size = len(tokenizer.word_index) + 1 # 단어 사전의 크기
embedding_dim = 100 # 임베딩 차원

model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(LSTM(128))
model.add(Dense(1, activation="sigmoid"))

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
model.summary()

# 모델 학습
model.fit(train_padded, train_labels, epochs=1, batch_size=64, validation_split=0.2)

# 모델 평가
model.evaluate(test_padded, test_labels)

In [ ]:
# 웹3 산업 관련 인용문에 대해 감정 분석 수행
quotes_padded = pad_sequences(tokenizer.texts_to_sequences(okt_quotes_morphs), maxlen=max_length, padding="post")
quotes_pred = model.predict(quotes_padded)
quotes_pred = np.round(quotes_pred).flatten() # 예측값을 반올림하여 0(부정) 또는 1(긍정)으로 변환

# 웹3 산업 관련 키워드에 대해 감정 분석 수행
keywords_padded = pad_sequences(tokenizer.texts_to_sequences(okt_keywords_morphs), maxlen=max_length, padding="post")
keywords_pred = model.predict(keywords_padded)
keywords_pred = np.round(keywords_pred).flatten() # 예측값을 반올림하여 0(부정) 또는 1(긍정)으로 변환

In [ ]:
# 인용문에 대한 감성 지수 계산
quotes_pos = np.sum(quotes_pred == 1) # 긍정 감성 점수
quotes_neg = np.sum(quotes_pred == 0) # 부정 감성 점수
quotes_score = (quotes_pos - quotes_neg) / (quotes_pos + quotes_neg) # 감성 지수

# 키워드에 대한 감성 지수 계산
keywords_pos = np.sum(keywords_pred == 1) # 긍정 감성 점수
keywords_neg = np.sum(keywords_pred == 0) # 부정 감성 점수
keywords_score = (keywords_pos - keywords_neg) / (keywords_pos + keywords_neg) # 감성 지수

# 감성 지수 출력
print("인용문의 감성 지수:", quotes_score)
print("키워드의 감성 지수:", keywords_score)